## Install OpenAI Library

In [19]:
pip install openai

In [20]:
pip install python-dotenv

## Import Library

In [21]:
import openai
import json
from dotenv import dotenv_values
config = dotenv_values(".env")

## Add API key

In [22]:
openai.api_key  = config["OPENAI_API_KEY"]

## Declare Functions

In [23]:
from IPython.display import Markdown, display

def display_colors1(colors, names):
    display(Markdown(" ".join(
        f'<div style="color: {color}">{name}: {chr(9608) * 5}</div>'
        for color, name in zip(colors, names)
    )))
    
# def display_colors2(colors):
#     display(Markdown(" ".join(
#         f'<div style="color: {color.code}">{color.name}: {chr(9608) * 5}</div>'
#         for color in colors
#     )))
    
def display_colors2(colors):
    display(Markdown(" ".join(
        f'<div style="color: {color["code"]}">{color["name"]}: {chr(9608) * 5}</div>'
        for color in colors
    )))


### 1st Prompt (GPT-3)

In [24]:
def get_and_render_prompt1(text):
    message=f"""
    You are a color palette generating assistant that responds to text prompt for color palettes.
    You should generate color palettes that fit the theme, mood, tone, or instructions in the prompt.
    
    Key Guidelines:
    - Avoid duplicating colors code and name within the same palette.
    - Keep the palettes within a range of 2 to 8 colors.
    - Use a JSON array to represent the color palettes, with each color code in hexadecimal format.
    
    Examples:
    - Q: Convert the following verbal description of a color palette into a list of colors: google brand
      A: ["#4285f4", "#34a853", "#fbbc05", "#ea4335"]

    - Q: Convert the following verbal description of a color palette into a list of colors: midnight pastels
      A: ["#B27DFA", "#6C5DC7", "#3E4AB0", "#2C3E8C", "#182F61", "#0B1E3E", "#080620"]

    Desired Format:
      - a JSON array of hexadecimal color codes

    Prompt: {text}
    Result:
    """

    completion = openai.Completion.create(
            model="text-davinci-003",
            prompt=message,
            max_tokens=100,
            temperature=0.8,
        )
    print(completion.choices[0].text)
    print("-----")
    print(completion)

    colors = json.loads(completion.choices[0].text.strip())
    display_colors1(colors)

### 2nd Prompt (GPT-3)

In [25]:
def get_and_render_prompt2(text):
    message=f"""
    Color Palette Generator

    Generate color palettes that match the given theme, mood, or instructions.

    Instructions:
    - Provide a verbal description of the color palette you want.
    - Avoid duplicating colors within the same palette.
    - Keep the palette size between 2 to 8 colors.

    Example Usage:
    - Instruction: Generate a color palette for a Google brand.
    - Output:"["#4285f4", "#34a853", "#fbbc05", "#ea4335"],["Google Blue", "Google Green", "Google Yellow", "Google Red"]"

    - Instruction: Generate a color palette for ocean pastels color palette.
    - Output:"["#83ADFC", "#53CBED", "#22D9DB", "#14D59C", "#0EC36F", "#2EAC8D", "#2A8FBC"], ["Sky Blue", "Light Blue", "Turquoise", "Mint Green", "Emerald Green", "Teal", "Navy Blue"]"

    Desired Response JSON Format:"["#color1", "#color2", ...], ["name1", "name2", ...]"

    Instruction: Generate a color palette for {text}

    Result:
    """

    completion = openai.Completion.create(
            model="text-davinci-003",
            prompt=message,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    print(completion.choices[0].text)
    print("-----")
    print(completion)

    result = completion.choices[0].text
    
    colors, names = result.split("], [")

    colors = f"""{colors}]"""
    names = f"""[{names}"""
    
    print(colors)
    print(names)
    
    colors = json.loads(colors)
    names = json.loads(names)
    
    display_colors1(colors, names)

### 3rd Prompt (GPT-3.5)

In [26]:
def get_and_render_prompt_chat1(text):
    messages = [
        {
            "role": "system",
            "content": """
            You're Color Palette Generator. 
            You'll generate color palettes that match the given theme, mood, or instructions. 
            You'll avoid duplicating colors and names within the same palette. 
            You'll keep the palette size between 2 to 8 colors.
            """
        },
        {
            "role": "user",
            "content": "Generate a color palette for a Google brand."
        },
        {
            "role": "assistant",
            "content": '["#4285f4", "#34a853", "#fbbc05", "#ea4335"], ["Google Blue", "Google Green", "Google Yellow", "Google Red"]'
        },
        {
            "role": "user",
            "content": "Generate a color palette for foresta."
        },
        {
            "role": "assistant",
            "content": '["#3F5F6D", "#2F4F4F", "#1B4F5F", "#006633", "#009933", "#33CC33", "#00CC00"], ["Midnight Blue", "Dark Slate Gray", "Dark Turquoise", "Forest Green", "Jungle Green", "Lime Green", "Green"]'
        },
        {
            "role": "user",
            "content": f"Generate a color palette for {text}"
        },
    ]

    completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    print(completion.choices[0].message.content)
    print("-----")
    print(completion)

    result = completion.choices[0].message.content.strip()
    
    colors, names = result.split("], [")

    colors = f"""{colors}]"""
    names = f"""[{names}"""
    
    print(colors)
    print(names)
    
    colors = json.loads(colors)
    names = json.loads(names)
    
    display_colors1(colors, names)

In [27]:
### 4th Prompt (GPT-3.5)

In [28]:
def get_and_render_prompt_chat2(text):
    messages = [
        {
            "role": "system",
            "content": """
            You're a colors palette generator assistant who must do your task without adding any comments or notes. 
            You'll generate color palettes that match the given theme, mood, or instructions. 
            You'll avoid duplicating color code or color name within the same response, but you can generate a new palette even if it is the same input.
            You must keep the palette size between 2 to 8 colors.
            You must return a JSON array, where each element follows this format: {"code": <color_code>, "name": <color_name>}
            If you recieve the same input, try generate a new colors palette again.
            If you recieve an empty input, just ramdomly generate a new beautiful colors palette.
            """
        },
        {
            "role": "user",
            "content": "Generate a color palette for: ocean breeze"
        },
        {
            "role": "assistant",
            "content": """
            [
              {"code": "#4ECDC4", "name": "Turquoise"},
              {"code": "#F7FFF7", "name": "Mint Cream"},
              {"code": "#9BC1BC", "name": "Opal"},
              {"code": "#5D5C61", "name": "Slate Gray"}
            ]
            """
        },
        {
            "role": "user",
            "content": f"Generate a color palette for: {text}"
        },
    ]

    completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    result = completion.choices[0].message.content
    print(result)
    print("-----")
    print(completion)

    palette = json.loads(result)
    print(palette)
    
    display_colors2(palette)
    

In [29]:
### 5th Prompt (GPT-3.5)

In [30]:
def get_and_render_prompt_chat3(text):
    messages = [
        {
            "role": "system",
            "content": """
            You're Color Palette Generator. 
            You'll generate color palettes that match the given theme, mood, or instructions. 
            You'll avoid duplicating colors and names within the same palette. 
            You'll keep the palette size between 2 to 8 colors.
            """
        },
        {
            "role": "user",
            "content": "Generate a color palette for: ocean breeze"
        },
        {
            "role": "assistant",
            "content": """
            [
                {"code": "#4ECDC4", "name": "Turquoise"},
                {"code": "#F7FFF7", "name": "Mint Cream"},
                {"code": "#9BC1BC", "name": "Opal"},
                {"code": "#5D5C61", "name": "Slate Gray"}
            ]
            """
        },
        {
            "role": "user",
            "content": "Generate a color palette for: thailand flag"
        },
        {
            "role": "assistant",
            "content": """
            [
                {"code": "#FF0000", "name": "Red"},
                {"code": "#FFFFFF", "name": "White"},
                {"code": "#0072C6", "name": "Cerulean Blue"}
            ]
            """
        },
        {
            "role": "user",
            "content": f"Generate a color palette for: {text}"
        },
    ]

    completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    result = completion.choices[0].message.content
    print(result)
    print("-----")
    print(completion)

    palette = json.loads(result)
    print(palette)
    
    display_colors2(palette)
    

## Insert Your Input

In [31]:
get_and_render_prompt2("ocean breeze")

In [32]:
get_and_render_prompt_chat1("foresta")

In [33]:
get_and_render_prompt_chat3("foresta")